## Skills requested in Google job posts

### Introduction

Which **language, skills, and experience** should we add to our toolbox for getting a job in Google? Google publishes all of their jobs at `careers.google.com`. Niyamat Ullah scraped all of the job data from that site by going every job page using a tool called **Selenium**, taking only the job title, location, responsibilities, minimum and preferred qualifications.

The data set, posted in the Kaggle site, contains posts for 1,250 jobs. The variables are:

* `company`: either Google or Youtube.

* `title`: the title of the job.

* `category`: the category of the job.

* `location`: The location of the job.

* `responsibilities`: the responsibilities for the job.

* `minqual`: minimum qualifications for the job.

* `prefqual`: preferred qualifications for the job.

### Importing the data

I load the CSV file that contains the data. The encoding is typically specified for files that contain text, to prevent problems with systems that do not use UTF-8 (Windows). This is probably not needed here, but it may save trouble in other cases.

In [1]:
import pandas as pd
google = pd.read_csv('https://raw.githubusercontent.com/iese-bad/' +
    'DataSci/master/Data/skills_google.csv', encoding='utf_8')

As usual, I check that the content of the file is what I expect. 

In [2]:
google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 7 columns):
company             1250 non-null object
title               1250 non-null object
category            1250 non-null object
location            1250 non-null object
responsibilities    1235 non-null object
minqual             1236 non-null object
prefqual            1236 non-null object
dtypes: object(7)
memory usage: 68.4+ KB


### Exploring the company

I start my exploratory analysis by the company, which, in most of the jobs, is Google.

In [3]:
google['company'].value_counts()

Google     1227
YouTube      23
Name: company, dtype: int64

### Exploring the job titles

Many different job titles are included in the data set. I extract the top-10 titles.

In [4]:
title = google['title']
title.unique().shape

(794,)

In [5]:
title.value_counts()[0:10]

Business Intern 2018                          35
MBA Intern, Summer 2018                       34
MBA Intern 2018                               28
BOLD Intern, Summer 2018                      21
Field Sales Representative, Google Cloud      17
Interaction Designer                          12
User Experience Researcher                     9
Partner Sales Engineer, Google Cloud           7
User Experience Design Intern, Summer 2018     7
Recruiter                                      7
Name: title, dtype: int64

The interns seems to dominate the picture, but, with 794 different titles, this quick view could be misleading. So, I check other possibilities. 

In [6]:
title.str.contains('Intern').sum()

187

In [7]:
title.str.contains('Sales').sum()

135

In [8]:
title.str.contains('Cloud').sum()

277

In [9]:
title.str.contains('Google Cloud').sum()

259

So far, the cloud dominates. To proceed more systematically, I extract a list of most frequent tokens. Previous to the extraction, I clean the data deleting all the expressions within parenthesis. To get that, I use the method `str.replace` with the **regular expression** ` [(].+[)]`. 

Some technicalities about this regular expression: 

1. In a regular expression, the parentheses are used for grouping pieces of text. To refer to the parentheses temselves, I use the square brackets.

2. The dot (.) stands for any character.

3. The plus symbol (+) is a **quantifier**, meaning any number of occurrences.

In [10]:
title_short = title.str.replace(' [(].+[)]', '')

Now, I split the composite titles, which have two parts separated by a comma. As shown by the following code line, `str.split` returns a data frame whose elements are lists of different lengths, so I will need a workaround to get a vector.

In [11]:
title_short.str.split(', ').head()

0                       [Google Cloud Program Manager]
1     [Supplier Development Engineer, Cable/Connector]
2    [Data Analyst, Product and Tools Operations, G...
3            [Developer Advocate, Partner Engineering]
4          [Program Manager, Audio Visual Deployments]
Name: title, dtype: object

The trick is to transform `title_short` into a single list. The expression `list(set(x))` produces a list with the same elements as `x`, but without duplicates.

In [12]:
defs = list(title_short.str.split(', '))
title_defs = defs[0]
for x in defs[1:]:
    title_defs = title_defs + x
len(list(set(title_defs)))

886

So, we have 886 unique title definitions. Next, I transform `title_defs` into a Pandas series, to be able to use the method `value_counts`. What do we find, in the end? That most of the jobs are for Google Cloud or for interns. The sales jobs seem to be very scattered, with many different title definitions.

In [13]:
title_defs = pd.Series(v for v in title_defs)
title_defs.value_counts()[:25]

Google Cloud                      206
Summer 2018                        76
Business Intern 2018               51
MBA Intern                         34
MBA Intern 2018                    32
Google Technical Services          31
Consumer Hardware                  27
Field Sales Representative         26
Google Cloud Platform              25
Product Marketing Manager          23
Program Manager                    22
Account Manager                    22
BOLD Intern                        21
Google Marketing Solutions         20
Associate Account Strategist       18
YouTube                            18
Interaction Designer               16
Google Professional Services       15
Software Engineer                  14
Large Customer Sales               13
Technical Program Manager          13
Technical Solutions Consultant     12
Recruiter                          12
User Experience Researcher         11
Google Play                        11
dtype: int64

### Exploring categories

I apply the same approach for categories. In the end, most of the jobs do not seem to call for techies.

In [14]:
google['category'].value_counts()

Sales & Account Management          168
Marketing & Communications          165
Finance                             115
Technical Solutions                 101
Business Strategy                    98
People Operations                    86
User Experience & Design             84
Program Management                   74
Partnerships                         60
Product & Customer Support           50
Legal & Government Relations         46
Administrative                       40
Sales Operations                     31
Software Engineering                 31
Hardware Engineering                 26
Real Estate & Workplace Services     25
Manufacturing & Supply Chain         16
Technical Infrastructure             11
Network Engineering                   6
Developer Relations                   5
Technical Writing                     5
IT & Data Management                  5
Data Center & Network                 2
Name: category, dtype: int64

### Exploring countries

To learn in which countries the posts are located, I extract the country from the location. The location has two or three components, separated by the string ', '. So I have to drop all the characters that come before the last occurrence of that string. I use the method `str.replace` and a regular expression which stands for all the strings to be suppressed, that is, for any string ending by a comma followed by white space. 

In [15]:
country = google['location'].str.replace('.+, +', '')
country.unique().shape

(49,)

There are 49 countries but, as we see next, most of the job requests are for US.

In [16]:
country.value_counts()[:10]

United States     638
Ireland            87
United Kingdom     62
Germany            54
Singapore          41
China              38
Australia          35
Japan              31
Taiwan             30
India              28
Name: location, dtype: int64

### Exploring responsibilities

To explore the content of responsibilities column, I put first everything in **lowercase**. Since there are missing values in this field of the data base, I have to get rid of the corresponding rows before applying the method `str.lower`.

In [17]:
resp = google['responsibilities']
resp.isnull().sum()
resp = resp.dropna()
resp = resp.str.lower()
resp.iloc[0]

'shape, shepherd, ship, and show technical programs designed to support the work of cloud customer engineers and solutions architects.\nmeasure and report on key metrics tied to those programs to identify any need to change course, cancel, or scale the programs from a regional to global platform.\ncommunicate status and identify any obstacles and paths for resolution to stakeholders, including those in senior roles, in a transparent, regular, professional and timely manner.\nestablish expectations and rationale on deliverables for stakeholders and program contributors.\nprovide program performance feedback to teams in product, engineering, sales, and marketing (among others) to enable efficient cross-team operations.'

Next, I extract the words. This will leave out **punctuation** and the **control character** '\n', which means new line, and is used to separate paragraphs. I use a regular expression that stands for any word with the method `str.findall` which returns a data frame whose elements are lists of different lengths. Each of these lists is a **bag of words**. To collect all the words together I need a workaround, similar to that used for `str.split`. 

In [18]:
terms = resp.str.findall('[a-z]+')
terms[0][:10]

['shape',
 'shepherd',
 'ship',
 'and',
 'show',
 'technical',
 'programs',
 'designed',
 'to',
 'support']

In [19]:
resp_terms = terms[0]
for x in terms[1:]:
    resp_terms = resp_terms + x
len(list(set(resp_terms)))

3824

In [20]:
resp_terms = pd.Series(v for v in resp_terms)
resp_terms.value_counts()[:25]

and          9457
to           4303
the          2668
of           2233
with         2182
for          1372
google       1292
in           1247
business     1218
a            1185
product       968
on            870
develop       779
teams         768
work          755
as            712
team          660
partners      633
technical     606
manage        596
customer      561
our           536
that          517
partner       516
support       489
dtype: int64

Most of these terms are **stopwords**, that is, words that do not contain relevant information (and, to, the, etc). The leading topics seem to be development, teams and partners. To get a better picture, I should continue the analysis by dropping the stopwords and merging **synonyms** (such as "team" and "teams"). I stop here.

### Exploring minimum qualifications

The analysis of the minimum qualifications follows the same lines.

In [21]:
minqual = google['minqual']
minqual.isnull().sum()
minqual = minqual.dropna()
minqual = minqual.str.lower()
minqual.iloc[0]

'ba/bs degree or equivalent practical experience.\n3 years of experience in program and/or project management in cloud computing, enterprise software and/or marketing technologies.'

In [22]:
terms = minqual.str.findall('[a-z]+')
minqual_terms = terms[0]
for x in terms[1:]:
    minqual_terms = minqual_terms + x
len(list(set(minqual_terms)))

1920

In [23]:
minqual_terms = pd.Series(v for v in minqual_terms)
minqual_terms.value_counts()[:25]

experience    3036
or            2478
in            2400
and           2304
a             1231
of            1110
equivalent    1063
degree        1059
practical      993
to             928
bs             879
ba             838
years          722
with           718
the            611
management     413
ability        363
field          341
related        321
working        313
program        305
as             292
english        286
speak          281
fluently       280
dtype: int64

Bad news here for the starters, experience is the main thing. I check how often are mentioned the leading programming languages. Not much.

In [24]:
minqual_terms.str.contains('sql').sum()

85

In [25]:
minqual_terms.str.contains('javascript').sum()

77

In [26]:
minqual_terms.str.contains('python').sum()

97

### Exploring preferred qualifications

I replicate the analysis for the preferred qualifications.

In [27]:
prefqual = google['prefqual']
prefqual.isnull().sum()
prefqual = prefqual.dropna()
prefqual = prefqual.str.lower()
prefqual.iloc[0]

'experience in the business technology market as a program manager in saas, cloud computing, and/or emerging technologies.\nsignificant cross-functional experience across engineering, sales, and marketing teams in cloud computing or related technical fields.\nproven successful program outcomes from idea to launch in multiple contexts throughout your career.\nability to manage the expectations, demands and priorities of multiple internal stakeholders based on overarching vision and success for global team health.\nability to work under pressure and possess flexibility with changing needs and direction in a rapidly-growing organization.\nstrong organization and communication skills.'

In [28]:
terms = prefqual.str.findall('[a-z]+')
prefqual_terms = terms[0]
for x in terms[1:]:
    prefqual_terms = prefqual_terms + x
len(list(set(prefqual_terms)))

3205

In [29]:
prefqual_terms = pd.Series(v for v in prefqual_terms)
prefqual_terms.value_counts()[:25]

and              6496
to               2991
in               2501
with             2419
experience       2308
ability          1856
of               1655
a                1654
skills           1461
or               1432
the              1379
management        720
business          680
demonstrated      640
as                582
excellent         570
work              549
communication     546
environment       520
technical         476
strong            472
analytical        446
working           440
cloud             395
data              394
dtype: int64

In [30]:
prefqual_terms.str.contains('sql').sum()

84

In [31]:
prefqual_terms.str.contains('javascript').sum()

66

In [32]:
prefqual_terms.str.contains('python').sum()

79